---
badges: true
categories:
- python
- kedro
date: '2024-02-08'
description: https://github.com/kedro-org/kedro/issues/3575
title: Investigation of the Kedro default node names
toc: true
hide: true
---

[Default node names are problematic #3575](https://github.com/kedro-org/kedro/issues/3575)

In [1]:
!kedro -V

kedro, version 0.18.11


In [3]:
%load_ext kedro.ipython
%reload_kedro default-node-name/

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


[02/08/24 15:51:58] INFO     Reached after_catalog_created hook                                        ]8;id=12272;file:///Users/Nok_Lam_Chan/dev/kedro/features/steps/test_plugin/plugin.py\plugin.py]8;;\:]8;id=269066;file:///Users/Nok_Lam_Chan/dev/kedro/features/steps/test_plugin/plugin.py#15\15]8;;\

                    INFO     Kedro project default_node_name                                        ]8;id=268323;file:///Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=71487;file:///Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#105\105]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=412030;file:///Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=453708;file:///Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#106\106]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=116680;file:///Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=551462;file:///Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#112\112]8;;\

In [4]:
n = pipelines["__default__"].nodes[0] # Getting the first node

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/qq/wpp9vnws3sqcv6v3td68qcpm0000gp/T/ipykernel_17905/2080366535.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/qq/wpp9vnws3sqcv6v3td68qcpm0000gp/T/ipykernel_17905/2080366535.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: list index out of range

In [5]:
pipelines["__default__"]

Pipeline([])